In [82]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
import numpy as np
import matplotlib.pyplot as plt

from concept_detection.interfaces.es import ES
from concept_detection.interfaces.wp import WP
from concept_detection.interfaces.db import DB

from concept_detection.test.courses.compare import *

In [84]:
db = DB()
wikified_course_descriptions = db.query_wikified_course_descriptions()

# Set of all keywords manually tagged by students in the course descriptions
keyword_list = list(set([result.keywords for course_id in wikified_course_descriptions for result in wikified_course_descriptions[course_id]]))

In [85]:
es = ES()
wp = WP()

In [55]:
def compare_keywords(keyword_list):
    sizes = [10, 3, 1]
    
    stats = {size: [] for size in sizes}
    for keywords in keyword_list:
        # Get results from wikipedia-api
        wp_results = wp.search(keywords)
        wp_page_ids = [result.page_id for result in wp_results]
        
        # Get results from elasticsearch
        es_results = es.search(keywords)
        es_page_ids = [result.page_id for result in es_results]

        for size in sizes:
            stats[size].append(confusion_stats(es_page_ids[:size], wp_page_ids[:size]))

    return stats

In [56]:
stats = compare_keywords(keyword_list[:10])

In [58]:
avgs = {
    size: np.mean([confusion['r'] for confusion in stats[size]])
    for size in sizes
}

In [60]:
avgs

{10: 0.25, 3: 0.23333333333333334, 1: 0.4}